In [37]:
from keras.layers import Input, Embedding, Dense, Lambda, merge, concatenate, Activation
from keras.models import Model
import keras.backend as K
import tensorflow as tf

## Model

In [38]:
# Vocab size
input_dim=1000
# Vector length
output_dim=512
# Sequence length
input_length=100
# Positice sample
n_positive=2
# Negative sample
n_negative=5

In [95]:
def antirectifier(x):
    sums = K.sum(x, axis=1, keepdims=False)
    normalisers = tf.count_nonzero(
        tf.count_nonzero(x, axis=2, keep_dims=False, dtype=tf.float32),
        axis=1, keep_dims=True, dtype=tf.float32)
    return sums / normalisers
def antirectifier_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 3  # only valid for 3D tensors
    return (shape[0], shape[-1],)

In [135]:
def cossim(x):
    dot_products = K.batch_dot(x[0], x[1], axes=[1,1])
    norm0 = tf.norm(x[0], ord=2, axis=1, keep_dims=True)
    norm1 = tf.norm(x[1], ord=2, axis=1, keep_dims=True)
    return dot_products / norm0 / norm1

In [136]:
main_input = Input(shape=(input_length,), dtype='int32', name='main_input')
pos_inputs = [Input(shape=(input_length,), dtype='int32', name='positive_input_{}'.format(i)) for i in range(n_positive)]
neg_inputs = [Input(shape=(input_length,), dtype='int32', name='negative_input_{}'.format(i)) for i in range(n_negative)]
embed = Embedding(output_dim=output_dim, input_dim=input_dim, input_length=input_length, name='embedding')
s = embed(main_input)
s_p = [embed(i) for i in pos_inputs]
s_n = [embed(i) for i in neg_inputs]
ave = Lambda(antirectifier, output_shape=antirectifier_output_shape, name='average')
ave_s = ave(s)
ave_s_p = [ave(i) for i in s_p]
ave_s_n = [ave(i) for i in s_n]
cos_p = [merge([ave_s, l], mode=lambda x: cossim(x), output_shape=(1,), name='p_cos_sim_{}'.format(i)) for i, l in enumerate(ave_s_p)]
cos_n = [merge([ave_s, l], mode=lambda x: cossim(x), output_shape=(1,), name='n_cos_sim_{}'.format(i)) for i, l in enumerate(ave_s_n)]
z = concatenate(cos_p + cos_n, axis=1, name='concatenate')
pred = Activation('softmax')(z)
model = Model(inputs=[main_input] + pos_inputs + neg_inputs, outputs=pred)
model.compile(optimizer='adam', loss='categorical_crossentropy')

/Users/Pikachu/.pyenv/versions/SiameseCBOW/lib/python3.4/site-packages/ipykernel_launcher.py:12: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if sys.path[0] == '':
/Users/Pikachu/.pyenv/versions/SiameseCBOW/lib/python3.4/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Users/Pikachu/.pyenv/versions/SiameseCBOW/lib/python3.4/site-packages/ipykernel_launcher.py:13: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  del sys.path[0]


In [137]:
import numpy as np
x = [np.ones((10,100))]*(1+n_positive+n_negative)
y = np.ones((10,(n_positive+n_negative)))

In [138]:
model.fit(x,y,epochs=1)

Epoch 1/1
10/10 [==============================] - 2s 155ms/step - loss: 13.6214


In [140]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
positive_input_0 (InputLayer)   (None, 100)          0                                            
__________________________________________________________________________________________________
positive_input_1 (InputLayer)   (None, 100)          0                                            
__________________________________________________________________________________________________
negative_input_0 (InputLayer)   (None, 100)          0                                            
__________________________________________________________________________________________________
negative_i